In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

df = pd.read_csv('2.5_CC_FRAUD.csv')

df['TRN_TYPE'] = df['TRN_TYPE'].apply(lambda x: 1 if x == 'FRAUD' else 0)


df['DOMAIN_FREQ'] = df.groupby('DOMAIN')['DOMAIN'].transform('count')
df['ZIPCODE_FREQ'] = df.groupby('ZIPCODE')['ZIPCODE'].transform('count')
E
df = pd.get_dummies(df, columns=['STATE'])

for time_col in ['TIME1', 'TIME2']:
    df[f'{time_col}_sin'] = np.sin(2 * np.pi * df[time_col]/24)
    df[f'{time_col}_cos'] = np.cos(2 * np.pi * df[time_col]/24)
df.drop(['TIME1', 'TIME2'], axis=1, inplace=True)

df.drop(['DOMAIN', 'ZIPCODE'], axis=1, inplace=True)

numerical_cols = ['VIS1', 'VIS2', 'XRN1', 'XRN2', 'XRN3', 'XRN4', 'XRN5',
                  'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'TRN_AMT',
                  'TOTAL_TRN_AMT', 'DOMAIN_FREQ', 'ZIPCODE_FREQ',
                  'TIME1_sin', 'TIME1_cos', 'TIME2_sin', 'TIME2_cos']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


X = df.drop('TRN_TYPE', axis=1)
y = df['TRN_TYPE']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42, shuffle=True
)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

model = XGBClassifier(
    scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]),
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    random_state=42
)

model.fit(X_train_res, y_train_res)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]


print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nAUC-ROC Score:", roc_auc_score(y_test, y_proba))

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test,y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.28      0.43     18518
           1       0.03      0.92      0.05       419

    accuracy                           0.29     18937
   macro avg       0.51      0.60      0.24     18937
weighted avg       0.97      0.29      0.42     18937


AUC-ROC Score: 0.7649155011662523
[[ 5093 13425]
 [   35   384]]
